# Notebook 0: extract proteomics data

This notebook extracts proteomics data via the Cohort Browser. This notebook extracts all ~1500 proteins measured in the first tranche of proteomics data released. This notebook also demonstrates how users can extract all proteomics data available from 500K participants or extract proteomics data associated with a specific phenotype or cohort.

The output is a dataframe that is samples x proteins.

### As-Is Software Disclaimer

This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

[MIT License](https://github.com/dnanexus/UKB_RAP/blob/main/LICENSE) applies to this notebook.

## JupyterLab app details

<b>Launch spec:</b>
- App name: JupyterLab
- Instance type: Spark cluster, 2 nodes
- Runtime: =~ 5 mins
- Cost ~= £0.47

### Dependencies

|Library |License|
|:------------- |:-------------|
|[pandas](https://pandas.pydata.org/) |[BSD-3](https://github.com/pandas-dev/pandas/blob/main/LICENSE)|

In [ ]:
# Import packages
# dxpy allows python to interact with the platform storage
# Note: This notebook is using spark since the size of the dataset we're extracting
# (i.e. the number of fields) is too large for a single node instance.
import dxpy
import pandas as pd
import subprocess
import glob
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [ ]:
output_dir = "/alee_example/"

In [ ]:
# Automatically discover dispensed dataset ID
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [ ]:
# Get project ID
project_id = dxpy.find_one_project()["id"]

In [ ]:
dataset = (":").join([project_id, dispensed_dataset_id])

In [ ]:
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

## Get field names

In [ ]:
path = os.getcwd()

In [ ]:
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

In [ ]:
field_names = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_0", "name"].values
)
print(len(field_names))

In [ ]:
field_names_str = [f"olink_instance_0.{f}" for f in field_names]
field_names_query = ",".join(field_names_str)

In [ ]:
# Export field name list to file for Table Exporter
# Alternatively, instead of using dx extract_dataset you can use the Table exporter app
# This list of field names can be used as input into the Table exporter app and then
# you can ignore running the remaining cells in this notebook

# file = open('field_names.txt','w')
# for item in field_names:
#    file.write(item+"\n")
# file.close()

In [ ]:
#!dx upload field_names.txt --destination /alee_example/

## Extract from full 500K dataset

In [ ]:
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "128m")

Due to the large number of proteins that we are extracting, `dx extract_dataset` fails using a single node instance. To resolve this we create an SQL query that we later use in spark.

In [ ]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

In [ ]:
cmd = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names_query,
    "--delimiter",
    ",",
    "--output",
    "extracted_data.sql",
    "--sql",
]
subprocess.check_call(cmd)

In [ ]:
with open("extracted_data.sql", "r") as file:
    retrieve_sql = ""
    for line in file:
        retrieve_sql += line.strip()

In [ ]:
temp_df = spark.sql(retrieve_sql.strip(";"))

In [ ]:
pdf = temp_df.toPandas()

In [ ]:
print(pdf.shape)
pdf.head()

## Extract from cohort

In [ ]:
# Discover cohort data
dispensed_control_id = list(
    dxpy.find_data_objects(
        typename="CohortBrowser",
        folder="/alee_example",
        name_mode="exact",
        name="ischaemic_control",
    )
)[0]["id"]

dispensed_case_id = list(
    dxpy.find_data_objects(
        typename="CohortBrowser",
        folder="/alee_example",
        name_mode="exact",
        name="ischaemic_cases",
    )
)[0]["id"]

In [ ]:
control_dataset = (":").join([project_id, dispensed_control_id])
case_dataset = (":").join([project_id, dispensed_case_id])

In [ ]:
cmd = [
    "dx",
    "extract_dataset",
    control_dataset,
    "--fields",
    field_names_query,
    "--delimiter",
    ",",
    "--output",
    "extracted_control_data.sql",
    "--sql",
]
subprocess.check_call(cmd)

In [ ]:
cmd = [
    "dx",
    "extract_dataset",
    case_dataset,
    "--fields",
    field_names_query,
    "--delimiter",
    ",",
    "--output",
    "extracted_case_data.sql",
    "--sql",
]
subprocess.check_call(cmd)

In [ ]:
with open("extracted_control_data.sql", "r") as file:
    retrieve_sql = ""
    for line in file:
        retrieve_sql += line.strip()

In [ ]:
temp_df = spark.sql(retrieve_sql.strip(";"))

In [ ]:
control_df = temp_df.toPandas()

In [ ]:
print(control_df.shape)
control_df.head()

In [ ]:
with open("extracted_case_data.sql", "r") as file:
    retrieve_sql = ""
    for line in file:
        retrieve_sql += line.strip()

In [ ]:
temp_df = spark.sql(retrieve_sql.strip(";"))

In [ ]:
case_df = temp_df.toPandas()

In [ ]:
print(case_df.shape)
case_df.head()